In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import shutil
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🖥️ Kullanılan cihaz:", device)

🖥️ Kullanılan cihaz: cpu


In [ ]:
# Yeni dataset klasörü oluştur
NEW_DATA_DIR = "/content/dataset"
os.makedirs(NEW_DATA_DIR, exist_ok=True)

# Sadece hastalık klasörlerini kopyala
for folder in ["Disease-Downy", "Disease-Powdery", "Healthy"]:
    src = f"/content/{folder}"
    dst = f"{NEW_DATA_DIR}/{folder}"
    if os.path.exists(src) and not os.path.exists(dst):
        shutil.copytree(src, dst)
        print(f"✅ {folder} kopyalandı")
    elif os.path.exists(dst):
        print(f"⚠️ {folder} zaten var, atlaniyor")

✅ Disease-Downy kopyalandı
✅ Disease-Powdery kopyalandı
✅ Healthy kopyalandı


In [ ]:
# Transform tanımla
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset yükle
dataset = datasets.ImageFolder(root=NEW_DATA_DIR, transform=transform)
print("\n📂 Bulunan klasörler:", dataset.classes)
print("📊 Toplam resim sayısı:", len(dataset))


📂 Bulunan klasörler: ['Disease-Downy', 'Disease-Powdery', 'Healthy']
📊 Toplam resim sayısı: 799


In [ ]:
# Train/Test split (70% train, 30% test)
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

print(f"\n✅ Train: {train_size} resim")
print(f"✅ Test: {test_size} resim")

# DataLoader oluştur
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print(f"📦 Train batch sayısı: {len(train_loader)}")
print(f"📦 Test batch sayısı: {len(test_loader)}")


✅ Train: 559 resim
✅ Test: 240 resim
📦 Train batch sayısı: 35
📦 Test batch sayısı: 15


In [ ]:
def train_model(model, model_name, epochs=15):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    print(f"\n🚀 {model_name} eğitimi başlıyor...")

    for epoch in range(epochs):
        model.train()
        running_loss = 0
        correct = 0
        total = 0

        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Her 10 batch'te bir ilerleme göster
            if (batch_idx + 1) % 10 == 0:
                print(f"  Epoch [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f"✅ Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

    print(f"🎉 {model_name} eğitimi tamamlandı!\n")
    return model

In [ ]:
def test_model(model, model_name):
    model.eval()
    all_preds = []
    all_labels = []

    print(f"\n🔍 {model_name} test ediliyor...")

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)

    print(f"\n{'='*60}")
    print(f"📊 {model_name} Test Sonuçları")
    print(f"{'='*60}")
    print(f"🎯 Accuracy: {accuracy*100:.2f}%")
    print("\n📋 Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=dataset.classes))
    print("\n🔢 Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))
    print(f"{'='*60}\n")

    return accuracy

In [ ]:
print("\n" + "="*60)
print("🤖 MODEL 1: RESNET18")
print("="*60)

resnet_model = models.resnet18(pretrained=True)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 3)
resnet_model = train_model(resnet_model, "ResNet18", epochs=15)

# Test et
resnet_accuracy = test_model(resnet_model, "ResNet18")

# Modeli kaydet
torch.save(resnet_model.state_dict(), "resnet18_model.pth")
print("💾 ResNet18 modeli kaydedildi: resnet18_model.pth")


🤖 MODEL 1: RESNET18
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 44.7M/44.7M [00:00<00:00, 224MB/s]



🚀 ResNet18 eğitimi başlıyor...
  Epoch [1/15], Batch [10/35], Loss: 0.6859
  Epoch [1/15], Batch [20/35], Loss: 0.4396
  Epoch [1/15], Batch [30/35], Loss: 0.3652
✅ Epoch 1/15 - Loss: 0.5817, Accuracy: 78.89%
  Epoch [2/15], Batch [10/35], Loss: 0.0834
  Epoch [2/15], Batch [20/35], Loss: 0.0947
  Epoch [2/15], Batch [30/35], Loss: 0.0327
✅ Epoch 2/15 - Loss: 0.1163, Accuracy: 98.21%
  Epoch [3/15], Batch [10/35], Loss: 0.0464
  Epoch [3/15], Batch [20/35], Loss: 0.0092
  Epoch [3/15], Batch [30/35], Loss: 0.0097
✅ Epoch 3/15 - Loss: 0.0434, Accuracy: 99.11%
  Epoch [4/15], Batch [10/35], Loss: 0.0186
  Epoch [4/15], Batch [20/35], Loss: 0.0098
  Epoch [4/15], Batch [30/35], Loss: 0.0152
✅ Epoch 4/15 - Loss: 0.0220, Accuracy: 99.82%
  Epoch [5/15], Batch [10/35], Loss: 0.0171
  Epoch [5/15], Batch [20/35], Loss: 0.0284
  Epoch [5/15], Batch [30/35], Loss: 0.0101
✅ Epoch 5/15 - Loss: 0.0229, Accuracy: 99.46%
  Epoch [6/15], Batch [10/35], Loss: 0.0121
  Epoch [6/15], Batch [20/35], Los

In [ ]:
print("\n" + "="*60)
print("🚀 MODEL 2: EFFICIENTNET-B0")
print("="*60)

efficientnet_model = models.efficientnet_b0(pretrained=True)
efficientnet_model.classifier[1] = nn.Linear(efficientnet_model.classifier[1].in_features, 3)
efficientnet_model = train_model(efficientnet_model, "EfficientNet-B0", epochs=15)

# Test et
efficientnet_accuracy = test_model(efficientnet_model, "EfficientNet-B0")

# Modeli kaydet
torch.save(efficientnet_model.state_dict(), "efficientnet_model.pth")
print("💾 EfficientNet modeli kaydedildi: efficientnet_model.pth")

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



🚀 MODEL 2: EFFICIENTNET-B0
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 81.7MB/s]



🚀 EfficientNet-B0 eğitimi başlıyor...
  Epoch [1/15], Batch [10/35], Loss: 0.9199
  Epoch [1/15], Batch [20/35], Loss: 0.8076
  Epoch [1/15], Batch [30/35], Loss: 1.0527
✅ Epoch 1/15 - Loss: 0.8503, Accuracy: 67.98%
  Epoch [2/15], Batch [10/35], Loss: 0.4671
  Epoch [2/15], Batch [20/35], Loss: 0.4270
  Epoch [2/15], Batch [30/35], Loss: 0.5639
✅ Epoch 2/15 - Loss: 0.5130, Accuracy: 86.40%
  Epoch [3/15], Batch [10/35], Loss: 0.4217
  Epoch [3/15], Batch [20/35], Loss: 0.2675
  Epoch [3/15], Batch [30/35], Loss: 0.2114
✅ Epoch 3/15 - Loss: 0.3275, Accuracy: 92.67%
  Epoch [4/15], Batch [10/35], Loss: 0.3083
  Epoch [4/15], Batch [20/35], Loss: 0.3801
  Epoch [4/15], Batch [30/35], Loss: 0.1550
✅ Epoch 4/15 - Loss: 0.2168, Accuracy: 95.89%
  Epoch [5/15], Batch [10/35], Loss: 0.2016
  Epoch [5/15], Batch [20/35], Loss: 0.1857
  Epoch [5/15], Batch [30/35], Loss: 0.0631
✅ Epoch 5/15 - Loss: 0.1569, Accuracy: 96.60%
  Epoch [6/15], Batch [10/35], Loss: 0.0961
  Epoch [6/15], Batch [20/3

In [ ]:
print("\n" + "="*60)
print("🏆 FINAL KARŞILAŞTIRMA")
print("="*60)
print(f"📊 ResNet18 Accuracy: {resnet_accuracy*100:.2f}%")
print(f"📊 EfficientNet-B0 Accuracy: {efficientnet_accuracy*100:.2f}%")

if resnet_accuracy > efficientnet_accuracy:
    print(f"\n🥇 Kazanan: ResNet18 ({(resnet_accuracy - efficientnet_accuracy)*100:.2f}% farkla)")
elif efficientnet_accuracy > resnet_accuracy:
    print(f"\n🥇 Kazanan: EfficientNet-B0 ({(efficientnet_accuracy - resnet_accuracy)*100:.2f}% farkla)")
else:
    print("\n🤝 İki model de aynı performansı gösterdi!")

print("="*60)


🏆 FINAL KARŞILAŞTIRMA
📊 ResNet18 Accuracy: 82.50%
📊 EfficientNet-B0 Accuracy: 88.33%

🥇 Kazanan: EfficientNet-B0 (5.83% farkla)


In [ ]:
# Dosyaları Google Drive'a kaydet (isteğe bağlı)
from google.colab import files

print("\n📥 Modeller indiriliyor...")
files.download('resnet18_model.pth')
files.download('efficientnet_model.pth')
print("✅ İndirme tamamlandı!")


📥 Modeller indiriliyor...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ İndirme tamamlandı!
